In [1]:
import os

In [2]:
train_img_dir = 'train_data/train_images/'
train_mask_dir = 'train_data/train_masks/'
test_img_dir = 'test_data/test_images/'

In [3]:
train_img_names = [x.split('.')[0] for x in os.listdir(train_img_dir)]

In [4]:
train_img_names[:5],len(train_img_names)

(['6419fd454b', '5a3c38104c', 'e63196d890', 'a48b9989ac', '623565cb2c'], 4000)

In [5]:
train_img_dict_i_to_names = dict()
train_img_dict_names_to_i = dict()
for i in range(len(train_img_names)):
    train_img_dict_i_to_names[i] = train_img_names[i]
    train_img_dict_names_to_i[train_img_names[i]] = i

In [6]:
from skimage.data import imread

In [7]:
train_img_shape = imread(train_img_dir + train_img_names[0]+'.png').shape
train_mask_shape = imread(train_mask_dir + train_img_names[0]+'.png').shape

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential           #層構造のモデルを定義するためのメソッド
from keras.layers import Dense, Activation          #Denseは層の生成メソッド、Activationは活性化関数を定義するためのメソッド
from keras.layers.convolutional import Conv2D,Conv2DTranspose
from keras.layers.pooling import MaxPool2D
from keras.layers import Flatten

from keras.utils import np_utils

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD,Adam

from keras.callbacks import TensorBoard
import keras.backend as K
from keras.layers import BatchNormalization,Input,Add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.models import Model, load_model, save_model

#from itertools import chain
from skimage.io import imread, imshow #, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model, save_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras import optimizers

from keras.preprocessing.image import array_to_img, img_to_array, load_img#,save_img

import time
t_start = time.time()

Using TensorFlow backend.


In [9]:
train_img = np.zeros((len(train_img_names), train_img_shape[0], train_img_shape[1], train_img_shape[2]))
train_mask = np.zeros((len(train_img_names), train_mask_shape[0], train_mask_shape[1]))

In [10]:
for i in range(len(train_img_names)):
    train_img[i] = i
    train_mask[i] = i
    train_img[i,:,:,:] = imread(train_img_dir + train_img_names[i]+'.png')
    train_mask[i,:,:] = imread(train_mask_dir + train_img_names[i]+'.png')


In [11]:
train_img.shape,train_mask.shape

((4000, 101, 101, 3), (4000, 101, 101))

In [12]:
train_img[50,:,:,0],train_mask[50,:,:]

(array([[172., 172., 171., ..., 154., 147., 149.],
        [172., 171., 171., ..., 156., 149., 150.],
        [171., 171., 170., ..., 157., 150., 149.],
        ...,
        [172., 171., 171., ..., 204., 189., 190.],
        [172., 171., 171., ..., 197., 175., 181.],
        [172., 171., 171., ..., 187., 161., 170.]]),
 array([[65535., 65535., 65535., ..., 65535., 65535., 65535.],
        [65535., 65535., 65535., ..., 65535., 65535., 65535.],
        [65535., 65535., 65535., ..., 65535., 65535., 65535.],
        ...,
        [65535., 65535., 65535., ...,     0.,     0.,     0.],
        [65535., 65535., 65535., ...,     0.,     0.,     0.],
        [65535., 65535., 65535., ...,     0.,     0.,     0.]]))

In [13]:
train_img_mono = np.zeros((len(train_img_names), train_img_shape[0], train_img_shape[1]))

In [14]:
train_img_mono = train_img[:,:,:,0]

In [15]:
train_img_mono.reshape(4000,101,101,1)

array([[[[151.],
         [154.],
         [161.],
         ...,
         [130.],
         [111.],
         [ 96.]],

        [[149.],
         [153.],
         [162.],
         ...,
         [118.],
         [105.],
         [ 93.]],

        [[147.],
         [155.],
         [165.],
         ...,
         [109.],
         [100.],
         [ 92.]],

        ...,

        [[108.],
         [136.],
         [156.],
         ...,
         [133.],
         [123.],
         [110.]],

        [[126.],
         [152.],
         [168.],
         ...,
         [119.],
         [110.],
         [101.]],

        [[145.],
         [169.],
         [179.],
         ...,
         [107.],
         [100.],
         [ 95.]]],


       [[[120.],
         [120.],
         [120.],
         ...,
         [121.],
         [126.],
         [128.]],

        [[120.],
         [120.],
         [120.],
         ...,
         [126.],
         [126.],
         [125.]],

        [[120.],
         [120.],
      

In [16]:
train_img_mono.shape

(4000, 101, 101)

In [17]:
train_mask_8bit = np.zeros((train_mask.shape[0],train_mask.shape[1],train_mask.shape[1]))

In [18]:
for i in range(len(train_img_names)):
    train_mask_8bit[i,:,:]= np.maximum(train_mask[i,:,:]/255-2,0)

In [19]:
train_mask_8bit[50,:,:]

array([[255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       ...,
       [255., 255., 255., ...,   0.,   0.,   0.],
       [255., 255., 255., ...,   0.,   0.,   0.],
       [255., 255., 255., ...,   0.,   0.,   0.]])

In [20]:
train = pd.read_csv('train_data/' + 'train.csv')

In [21]:
train.head(3)

,id,rle_mask
0,575d24d81d,NaN
1,a266a2a9df,5051 5151
2,75efad62c1,9 93 109 94 210 94 310 95 411 95 511 96 612 96...


In [22]:
train.shape

(4000, 2)

In [23]:
depths = pd.read_csv('train_data/' + 'depths.csv')

In [24]:
depths.head(3)

,id,z
0,4ac19fb269,306
1,1825fadf99,157
2,f59821d067,305


In [25]:
depths.shape

(22000, 2)

In [26]:
train = pd.merge(train, depths, on='id',how='left')

In [27]:
train.head(3)

,id,rle_mask,z
0,575d24d81d,NaN,843
1,a266a2a9df,5051 5151,794
2,75efad62c1,9 93 109 94 210 94 310 95 411 95 511 96 612 96...,468


In [28]:
train.shape

(4000, 3)

# 計算するんご


In [29]:
train.head(100)

,id,rle_mask,z
0,575d24d81d,NaN,843
1,a266a2a9df,5051 5151,794
2,75efad62c1,9 93 109 94 210 94 310 95 411 95 511 96 612 96...,468
3,34e51dba6a,48 54 149 54 251 53 353 52 455 51 557 50 659 4...,727
4,4875705fb0,1111 1 1212 1 1313 1 1414 1 1514 2 1615 2 1716...,797
5,782ae9b7e7,1 1815 1819 90 1920 81 2021 73 2122 64 2223 55...,677
6,9842f69f8d,NaN,907
7,aa94cfb806,1 28 102 28 203 29 304 30 405 32 506 33 607 34...,754
8,50d3073821,1 2121 9293 909,810
9,28f865caaa,NaN,147


In [22]:
train_mask_8bit.shape


(4000, 101, 101)

In [25]:
train_img_mono= train_img_mono.reshape(4000, 101,101,1)

In [26]:
train_img_mono.shape

(4000, 101, 101, 1)

In [27]:
from sklearn.model_selection import train_test_split
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(train_img_mono,train_mask_8bit, test_size=0.2, random_state=0)


In [65]:
X_1_train.shape

(3200, 101, 101, 1)

In [28]:
X_1_train = X_1_train.astype('float64')
X_1_test = X_1_test.astype('float64')

X_1_train /= 255
X_1_test /= 255

In [67]:
print(y_1_train)

[[[255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 [[255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  ...
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]]

 [[  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 ...

 [[  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 [[  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.

In [68]:
y_1_train.shape

(3200, 101, 101)

In [29]:
y_2_train=(y_1_train/255)
y_2_test=(y_1_test/255)

In [70]:
print(y_2_train)

[[[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  ...
  [1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]]]


In [30]:
y_2_train = y_2_train.reshape(3200, 101,101,1)
y_2_test = y_2_test.reshape(800, 101,101,1)

In [72]:
print(X_1_test.shape)

(800, 101, 101, 1)


In [73]:
print(y_2_test.shape)

(800, 101, 101, 1)


In [31]:
X_1_train /= 255.0
X_1_test /= 255.0

In [32]:
X_1_train = np.append(X_1_train, [np.fliplr(x) for x in X_1_train], axis=0)
y_2_train = np.append(y_2_train, [np.fliplr(y) for y in y_2_train], axis=0)

print(X_1_train.shape)
print(y_2_train.shape)


(6400, 101, 101, 1)
(6400, 101, 101, 1)


In [76]:
print(X_1_train)

[[[[0.00219915]
   [0.00221453]
   [0.00222991]
   ...
   [0.00236832]
   [0.0023837 ]
   [0.0023837 ]]

  [[0.00222991]
   [0.00222991]
   [0.00224529]
   ...
   [0.00233756]
   [0.00235294]
   [0.00236832]]

  [[0.00226067]
   [0.00224529]
   [0.00224529]
   ...
   [0.00230681]
   [0.00232218]
   [0.00233756]]

  ...

  [[0.00236832]
   [0.00233756]
   [0.0023837 ]
   ...
   [0.00226067]
   [0.00229143]
   [0.00226067]]

  [[0.00232218]
   [0.00235294]
   [0.00241446]
   ...
   [0.00233756]
   [0.00233756]
   [0.00230681]]

  [[0.00235294]
   [0.00239908]
   [0.00242983]
   ...
   [0.00235294]
   [0.00230681]
   [0.00226067]]]


 [[[0.00199923]
   [0.00172241]
   [0.0014456 ]
   ...
   [0.00118416]
   [0.00143022]
   [0.0016609 ]]

  [[0.00199923]
   [0.00183007]
   [0.00153787]
   ...
   [0.00107651]
   [0.00163014]
   [0.00161476]]

  [[0.00222991]
   [0.00196847]
   [0.00158401]
   ...
   [0.00130719]
   [0.00192234]
   [0.00167628]]

  ...

  [[0.00207612]
   [0.00158401]
   [0.0

In [77]:
X_1_train.shape

(6400, 101, 101, 1)

In [217]:
def build_multilayer_perceptron():
    
    model = Sequential()

    model.add(Conv2D(filters=64, input_shape=(101, 101,3), kernel_size=(4), strides=(1, 1), padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=64, kernel_size=(4), strides=(1, 1), padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=64, kernel_size=(4), strides=(1, 1), padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=128, kernel_size=(4), strides=(1, 1), padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('sigmoid'))
    model.add(Dense(10201))
    model.add(Activation('softmax'))
    
    return model

In [225]:
model = build_multilayer_perceptron()

NameError: name 'Input' is not defined

In [226]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_93 (Conv2D)           (None, 101, 101, 64)      3136      
_________________________________________________________________
max_pooling2d_84 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
activation_115 (Activation)  (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 50, 50, 64)        65600     
_________________________________________________________________
max_pooling2d_85 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
activation_116 (Activation)  (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 25, 25, 64)        65600     
__________

In [47]:
def get_iou_vector(A, B):
    A = np.squeeze(A) # new added 
    B = np.squeeze(B) # new added
    batch_size = A.shape[0]
    metric = []
    for batch in range(batch_size):
        t, p = A[batch]>0, B[batch]>0
        if np.count_nonzero(t) == 0 and np.count_nonzero(p) > 0:
            metric.append(0)
            continue
        if np.count_nonzero(t) >= 1 and np.count_nonzero(p) == 0:
            metric.append(0)
            continue
        if np.count_nonzero(t) == 0 and np.count_nonzero(p) == 0:
            metric.append(1)
            continue
        
        intersection = np.logical_and(t, p)
        union = np.logical_or(t, p)
        iou = (np.sum(intersection > 0)  )/ (np.sum(union > 0) )
        thresholds = np.arange(0.5, 1, 0.05)
        s = []
        for thresh in thresholds:
            s.append(iou > thresh)
        metric.append(np.mean(s))

    return np.mean(metric)

def my_iou_metric(label, pred):
    return tf.py_func(get_iou_vector, [label, pred>0.5], tf.float64)

In [232]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=False),
              metrics=[my_iou_metric])

In [182]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=[my_iou_metric])

In [233]:
history = model.fit(X_1_train, y_2_train,
                   batch_size=100,
                   epochs=5,
                   verbose=1,
                   validation_data=(X_1_test, y_2_test))

ValueError: Error when checking target: expected activation_120 to have 2 dimensions, but got array with shape (3200, 101, 101, 1)

In [50]:
def plot_history_loss(hist):
    # 損失値(Loss)の遷移のプロット
    plt.plot(hist.history['loss'],label="loss for training")
    plt.plot(hist.history['val_loss'],label="loss for validation")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.show()

def plot_history_acc(hist):
    # 精度(Accuracy)の遷移のプロット
    plt.plot(hist.history['acc'],label="loss for training")
    plt.plot(hist.history['val_acc'],label="loss for validation")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc='best')
    plt.ylim([0, 1])
    plt.show()

plot_history_loss(history)
plot_history_acc(history)

NameError: name 'history' is not defined

In [33]:
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [34]:
def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
    # 101 -> 50
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(input_layer)
    conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1, True)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(DropoutRatio/2)(pool1)

    # 50 -> 25
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2, True)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(DropoutRatio)(pool2)

    # 25 -> 12
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4, True)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(DropoutRatio)(pool3)

    # 12 -> 6
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    conv4 = residual_block(conv4,start_neurons * 8)
    conv4 = residual_block(conv4,start_neurons * 8, True)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(DropoutRatio)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 16)
    convm = residual_block(convm,start_neurons * 16, True)
    
    # 6 -> 12
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(DropoutRatio)(uconv4)
    
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = residual_block(uconv4,start_neurons * 8, True)
    
    # 12 -> 25
    #deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="valid")(uconv4)
    uconv3 = concatenate([deconv3, conv3])    
    uconv3 = Dropout(DropoutRatio)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = residual_block(uconv3,start_neurons * 4, True)

    # 25 -> 50
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
        
    uconv2 = Dropout(DropoutRatio)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = residual_block(uconv2,start_neurons * 2, True)
    
    # 50 -> 101
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    
    uconv1 = Dropout(DropoutRatio)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1, True)
    
    #uconv1 = Dropout(DropoutRatio/2)(uconv1)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    output_layer_noActi = Conv2D(1, (1,1), padding="same", activation=None)(uconv1)
    output_layer =  Activation('sigmoid')(output_layer_noActi)
    
    return output_layer

In [38]:
def get_iou_vector(A, B):
    batch_size = A.shape[0]
    metric = []
    for batch in range(batch_size):
        t, p = A[batch]>0, B[batch]>0
#         if np.count_nonzero(t) == 0 and np.count_nonzero(p) > 0:
#             metric.append(0)
#             continue
#         if np.count_nonzero(t) >= 1 and np.count_nonzero(p) == 0:
#             metric.append(0)
#             continue
#         if np.count_nonzero(t) == 0 and np.count_nonzero(p) == 0:
#             metric.append(1)
#             continue
        
        intersection = np.logical_and(t, p)
        union = np.logical_or(t, p)
        iou = (np.sum(intersection > 0) + 1e-10 )/ (np.sum(union > 0) + 1e-10)
        thresholds = np.arange(0.5, 1, 0.05)
        s = []
        for thresh in thresholds:
            s.append(iou > thresh)
        metric.append(np.mean(s))

    return np.mean(metric)

def my_iou_metric(label, pred):
    return tf.py_func(get_iou_vector, [label, pred>0.5], tf.float64)

def my_iou_metric_2(label, pred):
    return tf.py_func(get_iou_vector, [label, pred >0], tf.float64)

In [42]:
def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    gts = tf.reduce_sum(gt_sorted)
    intersection = gts - tf.cumsum(gt_sorted)
    union = gts + tf.cumsum(1. - gt_sorted)
    jaccard = 1. - intersection / union
    jaccard = tf.concat((jaccard[0:1], jaccard[1:] - jaccard[:-1]), 0)
    return jaccard

In [43]:
def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        def treat_image(log_lab):
            log, lab = log_lab
            log, lab = tf.expand_dims(log, 0), tf.expand_dims(lab, 0)
            log, lab = flatten_binary_scores(log, lab, ignore)
            return lovasz_hinge_flat(log, lab)
        losses = tf.map_fn(treat_image, (logits, labels), dtype=tf.float32)
        loss = tf.reduce_mean(losses)
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss

In [44]:
def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """

    def compute_loss():
        labelsf = tf.cast(labels, logits.dtype)
        signs = 2. * labelsf - 1.
        errors = 1. - logits * tf.stop_gradient(signs)
        errors_sorted, perm = tf.nn.top_k(errors, k=tf.shape(errors)[0], name="descending_sort")
        gt_sorted = tf.gather(labelsf, perm)
        grad = lovasz_grad(gt_sorted)
        loss = tf.tensordot(tf.nn.relu(errors_sorted), tf.stop_gradient(grad), 1, name="loss_non_void")
        return loss

    # deal with the void prediction case (only void pixels)
    loss = tf.cond(tf.equal(tf.shape(logits)[0], 0),
                   lambda: tf.reduce_sum(logits) * 0.,
                   compute_loss,
                   strict=True,
                   name="loss"
                   )
    return loss


In [57]:
def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = tf.reshape(scores, (-1,))
    labels = tf.reshape(labels, (-1,))
    if ignore is None:
        return scores, labels
    valid = tf.not_equal(labels, ignore)
    vscores = tf.boolean_mask(scores, valid, name='valid_scores')
    vlabels = tf.boolean_mask(labels, valid, name='valid_labels')
    return vscores, vlabels

def lovasz_loss(y_true, y_pred):
    y_true, y_pred = K.cast(K.squeeze(y_true, -1), 'int32'), K.cast(K.squeeze(y_pred, -1), 'float32')
    #logits = K.log(y_pred / (1. - y_pred))
    logits = K.log(y_pred / (1. - y_pred))
    loss = lovasz_hinge(logits, y_true, per_image = True, ignore = None)
    return loss

In [58]:
input_layer = Input((101, 101, 1))
output_layer = build_model(input_layer, 16,0.5)

model1 = Model(input_layer, output_layer)

c = optimizers.adam(lr = 0.01)
model1.compile(loss=lovasz_loss, optimizer=c, metrics=[my_iou_metric])


In [59]:
version = 5
basic_name = f'Unet_resnet_v{version}'
save_model_name = basic_name + '.model'
submission_file = basic_name + '.csv'

In [60]:
model_checkpoint = ModelCheckpoint(save_model_name,monitor='my_iou_metric', 
                                   mode = 'max', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='my_iou_metric', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

epochs = 10
batch_size = 32
history = model1.fit(X_1_train, y_2_train,
                    validation_data=[X_1_test, y_2_test], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[ model_checkpoint,reduce_lr], 
                    verbose=2)

Train on 6400 samples, validate on 800 samples
Epoch 1/10
 - 218s - loss: 1.0337 - my_iou_metric: 0.2327 - val_loss: 1.0004 - val_my_iou_metric: 0.3887

Epoch 00001: my_iou_metric improved from -inf to 0.23272, saving model to Unet_resnet_v5.model
Epoch 2/10
 - 197s - loss: 1.0019 - my_iou_metric: 0.3020 - val_loss: 1.0007 - val_my_iou_metric: 0.3887

Epoch 00002: my_iou_metric improved from 0.23272 to 0.30198, saving model to Unet_resnet_v5.model
Epoch 3/10
 - 197s - loss: 1.0017 - my_iou_metric: 0.3213 - val_loss: 1.0006 - val_my_iou_metric: 0.1314

Epoch 00003: my_iou_metric improved from 0.30198 to 0.32128, saving model to Unet_resnet_v5.model
Epoch 4/10


KeyboardInterrupt: 

In [ ]:
fig, (ax_loss, ax_score) = plt.subplots(1, 2, figsize=(15,5))
ax_loss.plot(history.epoch, history.history["loss"], label="Train loss")
ax_loss.plot(history.epoch, history.history["val_loss"], label="Validation loss")
ax_loss.legend()
ax_score.plot(history.epoch, history.history["my_iou_metric"], label="Train score")
ax_score.plot(history.epoch, history.history["val_my_iou_metric"], label="Validation score")
ax_score.legend()